In [ ]:
import numpy as np

rear_data = np.array(
    [
        # ignore since probably sensor noise
        # [80, 3],
        # [70, 3.25],
        # [60, 4],
        # [50, 4.75],
        # [40, 6],
        [30, 9.5],
        [25, 14],
        [20, 19.5],
        [18, 24],
        [16, 30.5],
        [14, 41],
        [12, 55],
        [10, 80],
        [8, 130],
        [6, 250],
       # ignore since this is capping out
       # [5, 255],
    ]
)

distance = rear_data[:,0]
reflectance = rear_data[:,1]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

# ── Dataset ──────────────────────────────────────────────────────────────────
x = reflectance
y = distance

x_fit = np.linspace(x.min(), x.max(), 300)

# ── Polynomial fit (degree 3) ─────────────────────────────────────────────────
degree = 3
poly_coeffs = np.polyfit(x, y, degree)
poly = np.poly1d(poly_coeffs)
y_poly = poly(x_fit)

res_poly = y - poly(x)
r2_poly = 1 - np.sum(res_poly**2) / np.sum((y - y.mean())**2)

eq_terms = []
for i, c in enumerate(poly_coeffs):
    p = degree - i
    if p > 1:    eq_terms.append(f"{c:+.3f}x^{p}")
    elif p == 1: eq_terms.append(f"{c:+.3f}x")
    else:        eq_terms.append(f"{c:+.3f}")
poly_eq = "y = " + " ".join(eq_terms)

# ── Logarithmic fit ───────────────────────────────────────────────────────────
log_coeffs = np.polyfit(np.log(x), y, 1)   # fit y = a·ln(x) + b
a_log, b_log = log_coeffs
y_log = a_log * np.log(x_fit) + b_log

res_log = y - (a_log * np.log(x) + b_log)
r2_log = 1 - np.sum(res_log**2) / np.sum((y - y.mean())**2)

log_eq = f"y = {a_log:+.3f}·ln(x) {b_log:+.3f}"

# ── Plotting ──────────────────────────────────────────────────────────────────
BG, GRID, SPINE = "#0f0f1a", "#1e1e30", "#2a2a40"
TICK, TEXT_C, EQ_C = "#8888aa", "#ccccff", "#aaffaa"
DATA_C, POLY_C, LOG_C = "#00d4ff", "#ff6b6b", "#a78bfa"

fig = plt.figure(figsize=(14, 9), facecolor=BG)
gs  = gridspec.GridSpec(2, 2, height_ratios=[3, 1], hspace=0.08, wspace=0.22)

ax_poly  = fig.add_subplot(gs[0, 0])
ax_log   = fig.add_subplot(gs[0, 1])
ax_rpoly = fig.add_subplot(gs[1, 0], sharex=ax_poly)
ax_rlog  = fig.add_subplot(gs[1, 1], sharex=ax_log)

for ax in [ax_poly, ax_log, ax_rpoly, ax_rlog]:
    ax.set_facecolor(BG)
    for sp in ax.spines.values(): sp.set_edgecolor(SPINE)
    ax.tick_params(colors=TICK, labelsize=9)
    ax.grid(color=GRID, linewidth=0.8, linestyle="--")

def scatter_data(ax):
    ax.scatter(x, y, color=DATA_C, s=38, alpha=0.85, zorder=3,
               label="Data", edgecolors="#005577", linewidths=0.5)

def eq_box(ax, text):
    ax.text(0.03, 0.06, text, transform=ax.transAxes, color=EQ_C,
            fontsize=8.5, fontfamily="monospace",
            bbox=dict(boxstyle="round,pad=0.4", facecolor="#1a1a2e", edgecolor=SPINE, alpha=0.92))

def residual_panel(ax, res, color):
    ax.axhline(0, color=color, linewidth=1.2, linestyle="--", alpha=0.7)
    ax.scatter(x, res, color=color, s=28, alpha=0.75, edgecolors="#005577", linewidths=0.5)
    ax.vlines(x, 0, res, color=color, alpha=0.25, linewidth=0.8)
    ax.set_ylabel("Residuals", color=TICK, fontsize=9)
    ax.set_xlabel("x", color=TICK, fontsize=10)

# Polynomial panel
scatter_data(ax_poly)
ax_poly.plot(x_fit, y_poly, color=POLY_C, linewidth=2.5, zorder=4, label=f"Degree-{degree} poly fit")
ax_poly.fill_between(x_fit, y_poly - 0.5, y_poly + 0.5, color=POLY_C, alpha=0.07)
ax_poly.set_title(f"Polynomial Fit  |  degree {degree}  |  R² = {r2_poly:.4f}", color=TEXT_C, fontsize=11, pad=10, fontfamily="monospace")
ax_poly.set_ylabel("y", color=TICK, fontsize=11)
ax_poly.legend(facecolor="#1a1a2e", edgecolor=SPINE, labelcolor=TEXT_C, fontsize=9)
eq_box(ax_poly, poly_eq)
plt.setp(ax_poly.get_xticklabels(), visible=False)

# Logarithmic panel
scatter_data(ax_log)
ax_log.plot(x_fit, y_log, color=LOG_C, linewidth=2.5, zorder=4, label="Logarithmic fit")
ax_log.fill_between(x_fit, y_log - 0.5, y_log + 0.5, color=LOG_C, alpha=0.07)
ax_log.set_title(f"Logarithmic Fit  |  y = a·ln(x)+b  |  R² = {r2_log:.4f}", color=TEXT_C, fontsize=11, pad=10, fontfamily="monospace")
ax_log.set_ylabel("y", color=TICK, fontsize=11)
ax_log.legend(facecolor="#1a1a2e", edgecolor=SPINE, labelcolor=TEXT_C, fontsize=9)
eq_box(ax_log, log_eq)
plt.setp(ax_log.get_xticklabels(), visible=False)

# Residual panels
residual_panel(ax_rpoly, res_poly, POLY_C)
residual_panel(ax_rlog,  res_log,  LOG_C)

plt.suptitle("Curve Fitting Comparison", color=TEXT_C, fontsize=14, fontfamily="monospace", y=1.01)
print(f"\nPolynomial:   {poly}\nR² = {r2_poly:.4f}")
print(f"\nLogarithmic:  y = {a_log:.4f}·ln(x) + {b_log:.4f}\nR² = {r2_log:.4f}")

In [ ]:
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# 1. Define the exponential decay function
def exponential_decay(x, a, k, b):
    """
    General form of an exponential decay function.
    a: amplitude
    k: decay constant (should be positive for decay, note the -k*x)
    b: y-offset (asymptote)
    """
    return a * np.exp(-k * x) + b

# 2. Create or load your data (example with synthetic data)
# Replace these with your actual x and y data
x_data = reflectance
# Generate data with noise: a=100, k=0.1, b=5
y_data = distance

# 3. Provide initial parameter guesses (crucial for good results)
# The default guess is [1, 1, 1], which often fails.
# Use reasonable estimates based on your data (e.g., initial y-value for 'a').
initial_guesses = [1, 1, 1]

# 4. Perform the curve fitting
# popt contains the optimized parameters, pcov is the covariance matrix
popt, pcov = curve_fit(exponential_decay, x_data, y_data, p0=initial_guesses)

# Extract the optimized parameters
a_fit, k_fit, b_fit = popt
print(f"Fitted parameters:")
print(f"Amplitude (a): {a_fit:.3f}")
print(f"Decay constant (k): {k_fit:.3f}")
print(f"Offset (b): {b_fit:.3f}")

# 5. Plot the original data and the fitted curve
plt.figure(figsize=(8, 5))
plt.scatter(x_data, y_data, label='Experimental Data', color='blue')

# Generate smooth x-values for the fitted curve
x_fit = np.linspace(min(x_data), max(x_data), 500)
y_fit = exponential_decay(x_fit, *popt)

plt.plot(x_fit, y_fit, 'r-', label=f'Fitted Curve: a={a_fit:.2f}, k={k_fit:.2f}, b={b_fit:.2f}')
plt.xlabel('X Data')
plt.ylabel('Y Data')
plt.title('Exponential Decay Fit using SciPy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
"""
double_exp_fit.py
=================
Fit a double exponential decay to arbitrary (t, y) data and plot the results.

Usage
-----
1. As a script with a CSV file:
       python double_exp_fit.py data.csv
   The CSV must have exactly two columns: time, signal (no header required;
   a header row is detected automatically).

2. Imported as a module:
       from double_exp_fit import fit_and_plot
       fit_and_plot(t_array, y_array,
                   xlabel="Time (ms)", ylabel="Voltage (mV)",
                   title="My Experiment", output="result.png")

3. Run with no arguments to fit built-in synthetic demo data:
       python double_exp_fit.py
"""

import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.optimize import curve_fit


# ── Model definitions ────────────────────────────────────────────────────────

def single_exp(t, A, tau, c):
    return A * np.exp(-t / tau) + c


def double_exp(t, A1, tau1, A2, tau2, c):
    return A1 * np.exp(-t / tau1) + A2 * np.exp(-t / tau2) + c


# ── Smart initial-guess & bounds from data ───────────────────────────────────

def _auto_p0_bounds(t, y):
    """Derive reasonable p0 and bounds from the data range."""
    t_range = t.max() - t.min()
    y_range = y.max() - y.min()
    y_min   = y.min()
    y_max   = y.max()

    # Amplitude guess: split signal into fast (~30 %) and slow (~70 %) fractions
    A1_guess   = 0.3 * y_range
    A2_guess   = 0.7 * y_range
    tau1_guess = 0.1 * t_range   # fast decay
    tau2_guess = 0.5 * t_range   # slow decay
    c_guess    = y_min

    p0 = [A1_guess, tau1_guess, A2_guess, tau2_guess, c_guess]

    eps = 1e-9
    bounds = (
        [0,   eps,            0,   eps,            y_min - abs(y_min)],
        [y_range * 2, t_range * 5, y_range * 2, t_range * 20, y_max],
    )
    return p0, bounds


def _auto_p0_single(t, y):
    t_range = t.max() - t.min()
    y_range = y.max() - y.min()
    return [y_range, 0.3 * t_range, y.min()]


# ── Core fit-and-plot function ───────────────────────────────────────────────

def fit_and_plot(
    t, y,
    xlabel="Time",
    ylabel="Signal",
    title="Double Exponential Decay — Curve Fitting Analysis",
    output="double_exp_fit.png",
    show=True,
):
    """
    Fit single- and double-exponential models to (t, y) and produce a
    publication-style plot.

    Parameters
    ----------
    t, y      : array-like  —  independent / dependent variable
    xlabel    : x-axis label
    ylabel    : y-axis label
    title     : figure title
    output    : path for the saved PNG (None = don't save)
    show      : call plt.show() if True

    Returns
    -------
    dict with keys: popt_single, popt_double, pcov_double,
                    rmse_single, rmse_double
    """
    t = np.asarray(t, dtype=float)
    y = np.asarray(y, dtype=float)

    # ── Fit single exp ───────────────────────────────────────────────────────
    p0_s = _auto_p0_single(t, y)
    try:
        popt_single, _ = curve_fit(single_exp, t, y, p0=p0_s, maxfev=20_000)
    except RuntimeError:
        print("Warning: single-exp fit did not converge; using initial guess.")
        popt_single = p0_s

    # ── Fit double exp ───────────────────────────────────────────────────────
    p0_d, bounds_d = _auto_p0_bounds(t, y)
    try:
        popt_double, pcov_double = curve_fit(
            double_exp, t, y, p0=p0_d, bounds=bounds_d, maxfev=50_000
        )
    except RuntimeError:
        print("Warning: double-exp fit did not converge; using initial guess.")
        popt_double = p0_d
        pcov_double = np.zeros((5, 5))

    # Ensure tau1 < tau2 (keep fast/slow labelling consistent)
    if popt_double[1] > popt_double[3]:
        popt_double[[0, 1, 2, 3]] = popt_double[[2, 3, 0, 1]]

    # ── Evaluate fits ────────────────────────────────────────────────────────
    t_fine = np.linspace(t.min(), t.max(), 600)
    y_single_fit = single_exp(t_fine, *popt_single)
    y_double_fit = double_exp(t_fine, *popt_double)

    res_single = y - single_exp(t, *popt_single)
    res_double = y - double_exp(t, *popt_double)
    rmse_single = np.sqrt(np.mean(res_single ** 2))
    rmse_double = np.sqrt(np.mean(res_double ** 2))
    perr = np.sqrt(np.diag(pcov_double))

    # ── Style ────────────────────────────────────────────────────────────────
    dark_bg  = '#0d0d14'
    panel_bg = '#13131f'
    grid_col = '#1e1e30'
    text_col = '#c8c8e0'
    accent1  = '#00e5ff'   # cyan  – double exp
    accent2  = '#ff6b6b'   # coral – single exp

    # ── Layout ───────────────────────────────────────────────────────────────
    fig = plt.figure(figsize=(12, 8), facecolor=dark_bg)
    gs = gridspec.GridSpec(
        3, 2, figure=fig, hspace=0.08, wspace=0.35,
        left=0.08, right=0.96, top=0.88, bottom=0.10,
        height_ratios=[3.5, 1.2, 1.2],
    )
    ax_main = fig.add_subplot(gs[0, :])
    ax_res1 = fig.add_subplot(gs[1, 0])
    ax_res2 = fig.add_subplot(gs[2, 0])
    ax_info = fig.add_subplot(gs[1:, 1])

    x_pad = (t.max() - t.min()) * 0.02

    for ax in [ax_main, ax_res1, ax_res2, ax_info]:
        ax.set_facecolor(panel_bg)
        ax.tick_params(colors=text_col, labelsize=8)
        for spine in ax.spines.values():
            spine.set_edgecolor(grid_col)
        ax.grid(True, color=grid_col, linewidth=0.5, alpha=0.8)

    # ── Main panel ───────────────────────────────────────────────────────────
    ax_main.scatter(t, y, color='white', s=18, alpha=0.75,
                    zorder=5, label='Observed data', edgecolors='none')
    ax_main.plot(t_fine, y_single_fit, color=accent2, linewidth=2,
                 alpha=0.85, linestyle='--',
                 label=f'Single exp  (RMSE={rmse_single:.4g})', zorder=6)
    ax_main.plot(t_fine, y_double_fit, color=accent1, linewidth=2.5,
                 label=f'Double exp  (RMSE={rmse_double:.4g})', zorder=7)

    y_comp1 = popt_double[0] * np.exp(-t_fine / popt_double[1])
    y_comp2 = popt_double[2] * np.exp(-t_fine / popt_double[3])
    ax_main.fill_between(t_fine, 0, y_comp1, alpha=0.08, color=accent1)
    ax_main.fill_between(t_fine, 0, y_comp2, alpha=0.05, color='#a78bfa')
    ax_main.plot(t_fine, y_comp1, color=accent1,   linewidth=1, alpha=0.4, linestyle=':')
    ax_main.plot(t_fine, y_comp2, color='#a78bfa', linewidth=1, alpha=0.4, linestyle=':')

    ax_main.set_ylabel(ylabel, color=text_col, fontsize=10, labelpad=8)
    ax_main.set_xlim(t.min() - x_pad, t.max() + x_pad)
    ax_main.legend(facecolor='#1a1a2e', edgecolor=grid_col,
                   labelcolor=text_col, fontsize=8.5, loc='upper right')

    # ── Residual panels ───────────────────────────────────────────────────────
    for ax, res, color, label in [
        (ax_res1, res_single, accent2, 'Single Exp Residuals'),
        (ax_res2, res_double, accent1, 'Double Exp Residuals'),
    ]:
        ax.axhline(0, color=color, linewidth=1, linestyle='--', alpha=0.6)
        ax.scatter(t, res, color=color, s=12, alpha=0.7, edgecolors='none')
        ax.set_ylabel('Residual', color=text_col, fontsize=8)
        ax.set_title(label, color=color, fontsize=8, pad=4)
        ax.set_xlim(t.min() - x_pad, t.max() + x_pad)

    ax_res1.tick_params(labelbottom=False)
    ax_res2.set_xlabel(xlabel, color=text_col, fontsize=9)

    # ── Info panel ────────────────────────────────────────────────────────────
    ax_info.axis('off')
    impr = (rmse_single - rmse_double) / rmse_single * 100 if rmse_single else 0
    info_lines = [
        ("DOUBLE EXPONENTIAL FIT",                            accent1,    11,  True),
        ("",                                                  text_col,    9,  False),
        ("f(t) = A₁·e^(−t/τ₁) + A₂·e^(−t/τ₂) + c",         '#a0a0c0', 8.5, False),
        ("",                                                  text_col,    9,  False),
        ("Fast Component",                                    '#a78bfa',   9,  True),
        (f"  A₁  = {popt_double[0]:.4g} ± {perr[0]:.3g}",   text_col,  8.5, False),
        (f"  τ₁  = {popt_double[1]:.4g} ± {perr[1]:.3g}",   text_col,  8.5, False),
        ("",                                                  text_col,    9,  False),
        ("Slow Component",                                    '#64b5f6',   9,  True),
        (f"  A₂  = {popt_double[2]:.4g} ± {perr[2]:.3g}",   text_col,  8.5, False),
        (f"  τ₂  = {popt_double[3]:.4g} ± {perr[3]:.3g}",   text_col,  8.5, False),
        (f"  c   = {popt_double[4]:.4g} ± {perr[4]:.3g}",   text_col,  8.5, False),
        ("",                                                  text_col,    9,  False),
        ("Model Comparison",                                  '#ffd54f',   9,  True),
        (f"  Single RMSE = {rmse_single:.4g}",                accent2,   8.5, False),
        (f"  Double RMSE = {rmse_double:.4g}",                accent1,   8.5, False),
        (f"  Improvement = {impr:.1f}%",                      '#69f0ae', 8.5, False),
    ]
    y_pos = 0.97
    for text_str, color, size, bold in info_lines:
        ax_info.text(0.05, y_pos, text_str, transform=ax_info.transAxes,
                     color=color, fontsize=size,
                     fontweight='bold' if bold else 'normal',
                     fontfamily='monospace', va='top')
        y_pos -= 0.063

    # ── Title ─────────────────────────────────────────────────────────────────
    fig.text(0.5, 0.95, title, ha='center', va='top', color='white',
             fontsize=13, fontweight='bold', fontfamily='monospace')

    if output:
        plt.savefig(output, dpi=150, bbox_inches='tight', facecolor=dark_bg)
        print(f"Plot saved → {output}")
    if show:
        plt.show()

    print(f"\nFitted parameters (double exp):")
    print(f"  A1={popt_double[0]:.6g},  tau1={popt_double[1]:.6g}")
    print(f"  A2={popt_double[2]:.6g},  tau2={popt_double[3]:.6g}")
    print(f"  offset={popt_double[4]:.6g}")
    print(f"\nRMSE — single: {rmse_single:.6g}  |  double: {rmse_double:.6g}  "
          f"({impr:.1f}% improvement)")

    return dict(popt_single=popt_single, popt_double=popt_double,
                pcov_double=pcov_double,
                rmse_single=rmse_single, rmse_double=rmse_double)


# ── CSV loader ───────────────────────────────────────────────────────────────

def load_csv(path):
    """
    Load a two-column CSV (time, signal).
    Automatically skips a header row if the first cell is non-numeric.
    Ignores blank lines and lines starting with '#'.
    """
    rows = []
    with open(path) as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            parts = line.split(',')
            if len(parts) < 2:
                parts = line.split()          # try whitespace delimiter
            try:
                rows.append((float(parts[0]), float(parts[1])))
            except ValueError:
                continue                      # skip header / bad rows
    if not rows:
        raise ValueError(f"No numeric data found in '{path}'.")
    arr = np.array(rows)
    return arr[:, 0], arr[:, 1]


# ── Demo data ────────────────────────────────────────────────────────────────

def _demo_data():
    np.random.seed(42)
    t = np.linspace(0, 5, 80)
    y = (3.5 * np.exp(-t / 0.5) + 1.8 * np.exp(-t / 2.5) + 0.2
         + np.random.normal(0, 0.12, len(t)))
    return t, np.maximum(y, 0)


fit_and_plot(reflectance, distance,
                     xlabel="Reflectance", ylabel="Distance (cm)",
                     output="double_exp_fit.png")


In [ ]:
A1=48.7536
tau1=6.19267
A2=16.9606
tau2=54.453
offset=6.05125
# When reflectance is below this value report max distance.
MIN_REFLECTANCE = 10.0
# Distance reported when reflectance is low.
MAX_DISTANCE = 100.0

def reflect_to_distance_cm(reflectance: float):
    if reflectance < MIN_REFLECTANCE:
        return MAX_DISTANCE
    return A1 * np.exp(-reflectance / tau1) + A2 * np.exp(-reflectance / tau2) + offset


print(reflect_to_distance_cm(5))
